In [4]:
class Grid:

    def __init__(self,x_len,y_len):
        self.x_len = x_len  # Hauteur (commence en haut à 0) le nombre de liste
        self.y_len = y_len  # Largeur (commence à gauche à 0) le nombre d'élement dans chaque liste
        self.grid = self.create_grid()

    # Method to create the board
    def create_grid(self)->list:
        
        grid=[]
        row = []
        
        for j in range(self.y_len):
            row.append("O")
        
        for i in range(self.x_len):
            grid.append(row.copy())
        
        return grid
    
    # Method to update the grid for each tour

    def update(self,fish_list,shark_list):
        
        self.grid = self.create_grid()

        for fish in fish_list:
            self.grid[fish.x][fish.y] = "F"

        for shark in shark_list:
            self.grid[shark.x][shark.y] = "S"

    def __repr__ (self):

        display_grid = " "
        for line in range(self.x_len):
            display_grid += "\n" 
            for column in range(self.y_len):
                display_grid += " " +self.grid[line][column]+ " "
        return display_grid


            

class Fish:

    nmb_of_rounds_for_reproduction = 0

    def __init__(self,x,y):
        self.x = x
        self.y = y

        self.old_x = x
        self.old_y = y

        self.counter = 0
        
    def reproduction(self, baby_list, fish_class):
        if self.counter == self.nmb_of_rounds_for_reproduction:
            # new baby fish
            baby = fish_class(self.old_x, self.old_y)
            
            baby_list.append(baby)
            
            self.counter = 0
    
    # Method for finding empty cells around
    def neighbor_coord (self,grid):
        
        up = (self.x-1,self.y) if self.x !=0 else (grid.x_len-1,self.y) 
        
        down = (self.x+1,self.y) if self.x !=grid.x_len-1 else (0,self.y)

        left = (self.x,self.y-1) if self.y !=0 else (self.x,grid.y_len-1)
        
        right = (self.x,self.y+1) if self.y !=grid.y_len-1 else (self.x,0)
        
        
        neighbor_coord_dictionary = {
            "up":up,
            "down":down,
            "right":right,
            "left":left
        }
        return neighbor_coord_dictionary
    
    # Method to move animals
    def move(self,grid,baby_fish_list):

            # A turn takes place for shark
            self.counter += 1

            # Looking for neighbot coordinates
            neighbor_coord_dict = self.neighbor_coord(grid)


            # sorting of neighbor coord in whether fill or empty cells
            empty_cells_list = []


            for i , j in neighbor_coord_dict.values():
                if grid.grid[i][j] == "O":
                    empty_cells_list.append((i,j))

            if len(empty_cells_list)>0:

                # Choice a neighbor fill cell as a target
                target = rd.choice(empty_cells_list)
                
                # Make move the shark to the target
                self.old_x = self.x
                self.old_y = self.y
                
                self.x = target[0]
                self.y = target[1]
                    
                
                # Sharks reproduction
                self.reproduction(baby_fish_list, Fish)
                
                # if self.counter == self.fish_nmb_of_rounds_for_reproduction:
                #     # crée un enfant
                #     baby_fish = Fish(self.old_x, self.old_y)

                #     baby_fish_list.append(baby_fish)

                #     self.counter = 0

class Shark(Fish):

    energy_by_fish = 0
    start_energy = 0
    
    def __init__(self,x,y):
        super().__init__(x,y)

        self.energy = self.start_energy
        

    def move(self, grid:Grid, fish_list, baby_shark_list, shark_list):

        # A turn takes place for shark
        self.counter += 1

        # Looking for neighbor coord 
        neighbor_coord_dict = self.neighbor_coord(grid)


        # sorting of neighbor coord in whether fill or empty cells
        cell_fish_list = []
        empty_cells_list = []


        for i , j in neighbor_coord_dict.values():
            if grid.grid[i][j] == "F":
                cell_fish_list.append((i,j))
            elif grid.grid[i][j] == "O":
                empty_cells_list.append((i,j))

        ########## If one of the nieghbors is a fish
        if len(cell_fish_list)>0:
            # Choice one of neighbors as a target
            target = rd.choice(cell_fish_list)
            # Move to the target
            self.old_x = self.x
            self.old_y = self.y
            
            self.x = target[0]
            self.y = target[1] 

            # Increase/Decresae shark energy
            self.energy += self.energy_by_fish
            self.energy -= 1

            # Take the target out of the list
            for fish in fish_list:
                if fish.x == target[0] and fish.y == target[1]:
                    fish_list.remove(fish)
            
            self.reproduction(baby_shark_list, Shark)
            
        elif len(empty_cells_list)>0:

            # Take a fish neighbor as a target
            target = rd.choice(empty_cells_list)
            # Make move the shark to the target
            self.old_x = self.x
            self.old_y = self.y
            
            self.x = target[0]
            self.y = target[1] 

            # Loose energy
            self.energy -= 1
            
            # Shark dies
            if self.energy == 0:
                shark_list.remove(self)
                
            
            # Shark reproduction
            self.reproduction(baby_shark_list, Shark)


        else:
            self.energy -= 1
            # Shark dies
            if self.energy == 0:
                shark_list.remove(self)


            
import random as rd



def create_animal_lists(x_len,y_len,fish_number,shark_number):
    my_set = set()
    while len(my_set) < fish_number + shark_number:
        x = rd.randint(0,x_len - 1)
        y = rd.randint(0,y_len - 1)
        coord = (x,y)
        my_set.add(coord)


    fish_list = []
    shark_list = []



    for coord in list(my_set)[0 : fish_number]:
        fish_list.append(Fish(coord[0],coord[1]))

    for coord in list(my_set)[fish_number :  ]:
        shark_list.append( Shark(coord[0],coord[1]))

    return fish_list, shark_list


# print(len(list(my_set)[fish_number : len(my_set)]))

In [5]:
######### INITIALISATION

Shark.nmb_of_rounds_for_reproduction = 15
Shark.energy_by_fish = 2
Shark.start_energy = 4
Fish.nmb_of_rounds_for_reproduction = 2

x_len_test = 5
y_len_test = 8

my_grid = Grid(x_len_test,y_len_test)

fish_list ,shark_list = create_animal_lists( x_len = x_len_test, y_len = y_len_test, fish_number = 7, shark_number = 2)

my_grid.update(fish_list, shark_list)

print("----Initialisation -----")
print("Sharks", len(shark_list))
print("Fishes", len(fish_list))
print(my_grid)


###########The Rounds
game_playing = True

round_nb = 0
while game_playing:
    baby_shark_list = []

    for shark in shark_list:
        shark.move(my_grid,fish_list, baby_shark_list, shark_list)
        jonction_shark = []
        jonction_shark.extend(shark_list)
        jonction_shark.extend(baby_shark_list) 
        my_grid.update(fish_list,jonction_shark)
        # print("---- Next Shark -----")
        # print("shark-number",len(jonction_shark))
        # print("fish_number",len(fish_list))
        # print(my_grid)

    shark_list.extend(baby_shark_list)


    baby_fish_list=[]
    for fish in fish_list:  
        fish.move(my_grid, baby_fish_list)
        jonction_fish = []
        jonction_fish.extend(fish_list)
        jonction_fish.extend(baby_fish_list) 
        my_grid.update(jonction_fish, fish_list)
        # print("---- Next Fish -----")
        # print("shark_number", len(shark_list))
        # print("fish_number", len(jonction_fish))
        # print(my_grid)


    fish_list.extend(baby_fish_list)

    print("------ Round n -------", round_nb)
    print("Sharks",len(shark_list))
    print("Fishes",len(fish_list))
    print(my_grid)


    round_nb +=1
    ### End game analysis:

    if len(fish_list) == 0:
        print("Tous les poissons sont morts, le jeu est fini")
        print (" Tour ", round_nb)
        game_playing = False
    elif len(shark_list) == 0:
        print("Tous les requins sont morts")
        print (" Tour ", round_nb)
        game_playing = False
    elif round_nb == 150:
        print("Vous avez atteint le tour 150, cest un bel équilibre")
        game_playing = False

----Initialisation -----
Sharks 2
Fishes 7
 
 F  O  O  O  O  O  O  O 
 O  O  O  O  O  O  S  O 
 F  O  O  O  F  O  O  O 
 O  F  O  F  O  O  O  O 
 O  O  F  F  O  O  O  S 
------ Round n ------- 0
Sharks 2
Fishes 7
 
 O  O  O  S  O  O  O  O 
 S  O  O  O  O  O  O  O 
 O  S  O  S  O  O  O  O 
 O  O  O  O  S  O  O  O 
 S  S  O  O  O  O  O  O 
------ Round n ------- 1
Sharks 2
Fishes 14
 
 O  O  S  F  O  O  O  O 
 F  S  O  S  O  O  O  O 
 S  F  O  F  O  O  O  O 
 S  O  O  O  F  S  O  O 
 F  F  S  O  O  O  O  O 
------ Round n ------- 2
Sharks 2
Fishes 14
 
 S  S  S  O  O  O  O  O 
 O  O  S  S  S  O  O  O 
 S  O  S  O  O  O  O  O 
 S  S  O  O  O  O  O  S 
 O  O  O  S  S  S  O  O 
------ Round n ------- 3
Sharks 1
Fishes 27
 
 F  F  F  O  S  O  O  O 
 S  S  F  F  F  O  O  O 
 F  S  F  S  O  O  O  S 
 F  S  S  S  S  O  O  F 
 S  S  S  F  F  F  S  O 
------ Round n ------- 4
Sharks 1
Fishes 26
 
 S  S  S  S  O  S  O  O 
 O  S  S  O  S  S  O  S 
 O  S  O  S  S  O  O  S 
 S  S  S  S  O  S  S  O 
 

In [6]:
dictionnaire_coord_voisin = {
    "up":1,
    "down":2,
    "right":3,
    "left":4
}


for i,j in dictionnaire_coord_voisin.items():
    print(i,j)

up 1
down 2
right 3
left 4
